<div class="alert alert-success" role="alert">
  <h1 class="alert-heading">Análisis Vibracional</h1>
  <h3 class="alert-heading">Prof. Enrique Mejía Ospino, emejia@uis.edu.co</h3>
  <h4 class="alert-heading">Escuela de Química</h4>
  <h4 class="alert-heading">Universidad Industrial de Santander</h4>
  </div>

**<font color=blue> Vamos  atilizar el modulo *PSI4* para desarrollar algunos ejercicios de cálculo mecánico-cuántico. En este Cuaderno de Jupyter realizaremos análisis vibracional de algunas moleculas sencillas.** 

In [1]:
# Importamos modulos ===============================================================
import psi4
import numpy as np  
import matplotlib.pyplot as plt
import h5py
import fortecubeview # Visualizar moléculas
%matplotlib inline

In [2]:
# Inicialización Psi4 =================================================================

psi4.core.clean()
psi4.core.clean_options()
psi4.set_memory('2000 MB')  # Can make this much larger on Seawulf, each compute node has more than 100 GB RAM
psi4.set_num_threads(4)    # Can make this much larger on Seawulf, each compute node can support 24 threads. 
                          # But it doesn't help much for small molecules...
psi4.core.set_output_file('./Data/OCS_bent.dat', False) 
# this command sets psi4's output to a file. Comment this line out if you want to see the output on the terminal.

In [3]:
# Geometría de la molécula de OCS ============================================
OCS_120 = psi4.geometry("""
 o
 c    1 co2     
 s    2 sc3         1 sco3      
 
co2=        1.380000
sc3=        1.780000
sco3=       120.000
""")
OCS_120.reset_point_group('c1') # simetría!

In [4]:
# Inicialización de los cálculos ================================================

metodo1 = 'scf/3-21G'  # Métodos de cálculos.
metodo2 = 'scf/6-311G*'

E0_120_1 = psi4.energy(metodo1, molecule = OCS_120)  # cálculo de energía usando metodo1
E0_120_2 = psi4.energy(metodo2, molecule = OCS_120)  # cálculo de energía usando metodo2

# Optimize with constrained bend angle.
psi4.set_module_options('optking',{'frozen_bend': '1 2 3'}) # This command fixes the bend angle of the OCS molecule.
Eopt_120, wfn_opt120 = psi4.optimize(metodo2, molecule = OCS_120, return_wfn = True) # optimize geometry of molecule with previously defined constraint, return wave function to make .molden file.
psi4.driver.molden(wfn_opt120, './Data/OCS_120.molden') #write constrained optimzation results to a .molden file.

# Now turn optimization constraint off and reoptimize to find the global energy minimum
psi4.core.clean_options() # turn off frozen bend and any other options
Eopt, wfn_opt = psi4.optimize(metodo2, molecule = OCS_120, return_wfn = True)

# Impresión de Resultados =================================================================
print('Energías en las posiciones iniciales con los dos métodos:')
print()
print('Energía con el ' + metodo1 + ' es ' + str(E0_120_1) + ' Hatrees')
print('Energía con el ' + metodo2 + ' es ' + str(E0_120_2) + ' Hatrees')
print()
print('Energía después de la optimización con ' + metodo2 + ' is ' + str(Eopt_120) + ' Hatrees')
print('Energía después de la optimización con ' + metodo2 + ' is ' + str(Eopt) + ' Hatrees')

Optimizer: Optimization complete!
Optimizer: Optimization complete!
Energías en las posiciones iniciales con los dos métodos:

Energía con el scf/3-21G es -507.50638932058047 Hatrees
Energía con el scf/6-311G* es -510.1193187715539 Hatrees

Energía después de la optimización con scf/6-311G* is -510.2107669863993 Hatrees
Energía después de la optimización con scf/6-311G* is -510.31252290663014 Hatrees


In [ ]:
# Save data to a spreadsheet using Pandas ==========================================
#
#  d = {'Details': [method1, method2, 'Global opt.'], 'Energies': [E0_1, E0_2, Eopt]} # create dictionary to load into Pandas dataframe.
#  df = pd.DataFrame(data = d)     # create Pandas dataframe
#  df.to_csv('Lab1starter_OCS_results.csv')  # save dataframe to .csv spreadsheet file.
 
# Or of course you can also extract the results from the raw .dat file
# generated by Psi4